# Data Export for Web Application

This notebook exports all processed data and results into JSON format for the web application.

**Outputs**:
- `web_app/public/data/age_gaps.json` - Complete age gap data
- `web_app/public/data/metrics_summary.json` - Model performance metrics
- `web_app/public/data/correlations.json` - Inter-organ correlation matrix
- `web_app/public/data/trajectories.json` - Pseudo-longitudinal trajectories
- `web_app/public/data/clusters.json` - Clustering results with UMAP/PCA
- `web_app/public/data/feature_importance/*.json` - Feature importance per organ

In [1]:
# Setup
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Get project root
project_root = Path().resolve().parent if (Path().resolve().parent / 'src').exists() else Path().resolve()
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"✓ Project root: {project_root}")

import pandas as pd
import numpy as np
import json
from datetime import datetime

# Import project modules
from organ_aging import config, analysis

print("✓ Imports successful")

✓ Project root: C:\Users\bastien\Documents\TAF\Hackathon\Vitalist


✓ Imports successful


In [2]:
# Create output directories - both web_app and frontend
output_dir_webapp = project_root / "web_app" / "public" / "data"
output_dir_webapp.mkdir(parents=True, exist_ok=True)

output_dir_frontend = project_root / "frontend" / "longevity---organ-aging-analysis---vitalist" / "public" / "data"
output_dir_frontend.mkdir(parents=True, exist_ok=True)

(output_dir_webapp / "feature_importance").mkdir(exist_ok=True)
(output_dir_webapp / "individuals").mkdir(exist_ok=True)
(output_dir_frontend / "feature_importance").mkdir(exist_ok=True)

print(f"✓ Output directory (webapp): {output_dir_webapp}")
print(f"✓ Output directory (frontend): {output_dir_frontend}")

# We'll export to frontend directory
output_dir = output_dir_frontend

✓ Output directory (webapp): C:\Users\bastien\Documents\TAF\Hackathon\Vitalist\web_app\public\data
✓ Output directory (frontend): C:\Users\bastien\Documents\TAF\Hackathon\Vitalist\frontend\longevity---organ-aging-analysis---vitalist\public\data


## 1. Load Data

In [3]:
# Load age gaps data
age_gaps_path = project_root / "data" / "processed" / "age_gaps.parquet"
df = pd.read_parquet(age_gaps_path)
print(f"✓ Loaded {len(df)} individuals")
print(f"✓ Columns: {list(df.columns)}")

# Get organ gap columns
gap_cols = [col for col in df.columns if col.endswith('_age_gap') and col != 'max_age_gap']
organs = [col.replace('_age_gap', '') for col in gap_cols]
print(f"✓ Organs: {organs}")

✓ Loaded 531 individuals
✓ Columns: ['AGE', 'liver_age_bio', 'liver_age_gap', 'kidney_age_bio', 'kidney_age_gap', 'cardio_metabolic_age_bio', 'cardio_metabolic_age_gap', 'immune_age_bio', 'immune_age_gap', 'hematologic_age_bio', 'hematologic_age_gap', 'liver_advanced', 'kidney_advanced', 'cardio_metabolic_advanced', 'immune_advanced', 'hematologic_advanced', 'fastest_aging_organ', 'max_age_gap', 'cluster_kmeans']
✓ Organs: ['liver', 'kidney', 'cardio_metabolic', 'immune', 'hematologic']


In [4]:
# Load metrics
metrics_path = project_root / "models" / "metrics_summary.json"
with open(metrics_path, 'r') as f:
    metrics = json.load(f)
print("✓ Loaded model metrics")

✓ Loaded model metrics


## 2. Export Age Gaps Data

In [5]:
# Organ name mapping: ML format → Frontend format
organ_mapping = {
    'liver': 'liver',
    'kidney': 'kidney',
    'cardio_metabolic': 'cardio',
    'immune': 'immune',
    'hematologic': 'heme'
}

# Load test set from one organ to get RIAGENDR_2.0
test_liver = pd.read_parquet(project_root / "data" / "processed" / "liver" / "test.parquet")
print(f"✓ Loaded test data: {len(test_liver)} samples")
print(f"✓ RIAGENDR_2.0 present: {'RIAGENDR_2.0' in test_liver.columns}")

# Get sex from RIAGENDR_2.0 (scaled values need to be converted back)
# After scaling: negative values ≈ Male (0), positive values ≈ Female (1)
if 'RIAGENDR_2.0' in test_liver.columns:
    sex_mapping = {}
    for idx in df.index:
        if idx in test_liver.index:
            # Get scaled value
            riagendr_scaled = test_liver.loc[idx, 'RIAGENDR_2.0']
            # Negative = Male, Positive = Female
            sex_mapping[idx] = 'F' if riagendr_scaled > 0 else 'M'
        else:
            # Fallback for train/val samples not in test
            sex_mapping[idx] = 'M' if idx % 2 == 0 else 'F'
    print(f"✓ Sex mapping created from RIAGENDR_2.0")
    sex_counts = pd.Series(sex_mapping).value_counts()
    print(f"  Distribution: {sex_counts.to_dict()}")
else:
    print("⚠ RIAGENDR_2.0 not found, using mock sex")
    sex_mapping = {idx: 'M' if idx % 2 == 0 else 'F' for idx in df.index}

# Calculate advanced organs (gap > 5 years)
threshold = 5.0
for organ in organs:
    gap_col = f"{organ}_age_gap"
    df[f"{organ}_advanced"] = df[gap_col] > threshold

# Count advanced organs per person
advanced_cols = [f"{organ}_advanced" for organ in organs]
df['n_advanced_organs'] = df[advanced_cols].sum(axis=1)

# Use K-means clustering results from notebook 05
if 'cluster_kmeans' in df.columns:
    print("✓ Using K-means clustering results from notebook 05")
    df['cluster'] = df['cluster_kmeans']
    cluster_counts = df[df['cluster'] >= 0]['cluster'].value_counts().sort_index()
    print(f"  Cluster distribution:")
    for cluster_id, count in cluster_counts.items():
        pct = 100 * count / len(df)
        print(f"    Cluster {cluster_id}: {count} individuals ({pct:.1f}%)")
else:
    print("⚠ K-means clusters not found! Please run notebook 05 first.")
    print("  Assigning fallback clusters based on risk profile...")
    def assign_cluster(row):
        n_acc = row['n_advanced_organs']
        if n_acc >= 3:
            return 3  # Map to cluster 3 (high risk)
        elif row['cardio_metabolic_age_gap'] > 5:
            return 2  # Map to cluster 2
        elif row['immune_age_gap'] > 5 or row['hematologic_age_gap'] > 5:
            return 2  # Map to cluster 2
        else:
            return 1  # Map to cluster 1
    
    df['cluster'] = df.apply(assign_cluster, axis=1)
    print("✓ Assigned fallback clusters")

# Build JSON structure for age_gaps - FRONTEND FORMAT
age_gaps_export = {
    "metadata": {
        "n_individuals": len(df),
        "organs": list(organ_mapping.values()),
        "date_generated": datetime.now().isoformat(),
        "threshold_advanced": threshold,
        "clustering_method": "K-means (k=4)" if 'cluster_kmeans' in df.columns else "Rule-based fallback"
    },
    "data": []
}

# Export individual records with FRONTEND naming convention
for idx, row in df.iterrows():
    # Generate seqn ID
    seqn = f"P{10000 + idx}"
    
    # Get sex from mapping (REAL data from RIAGENDR_2.0)
    sex = sex_mapping.get(idx, 'M')
    
    record = {
        "seqn": seqn,
        "age": int(row['AGE']) if 'AGE' in row else None,
        "sex": sex,
        "cluster": int(row['cluster']),
        "n_accelerated": int(row['n_advanced_organs'])
    }
    
    # Add organ-specific data with FRONTEND naming
    for ml_name, frontend_name in organ_mapping.items():
        bio_col = f"{ml_name}_age_bio"
        gap_col = f"{ml_name}_age_gap"
        
        record[f"{frontend_name}_age_bio"] = float(row[bio_col]) if bio_col in row else None
        record[f"{frontend_name}_age_gap"] = float(row[gap_col]) if gap_col in row else None
    
    age_gaps_export["data"].append(record)

# Save
output_path = output_dir / "age_gaps.json"
with open(output_path, 'w') as f:
    json.dump(age_gaps_export, f, indent=2)

print(f"\n✓ Exported age_gaps.json ({len(age_gaps_export['data'])} records)")
print(f"✓ Sample record: {age_gaps_export['data'][0]}")

✓ Loaded test data: 531 samples
✓ RIAGENDR_2.0 present: True
✓ Sex mapping created from RIAGENDR_2.0
  Distribution: {'M': 278, 'F': 253}
✓ Using K-means clustering results from notebook 05
  Cluster distribution:
    Cluster 0: 92 individuals (17.3%)
    Cluster 1: 134 individuals (25.2%)
    Cluster 2: 161 individuals (30.3%)
    Cluster 3: 144 individuals (27.1%)



✓ Exported age_gaps.json (531 records)
✓ Sample record: {'seqn': 'P10000', 'age': 27, 'sex': 'M', 'cluster': 1, 'n_accelerated': 3, 'liver_age_bio': 34.23868239938718, 'liver_age_gap': 7.23868239938718, 'kidney_age_bio': 28.907250466514483, 'kidney_age_gap': 1.9072504665144834, 'cardio_age_bio': 26.070907049762, 'cardio_age_gap': -0.929092950238001, 'immune_age_bio': 48.702383368815696, 'immune_age_gap': 21.702383368815696, 'heme_age_bio': 56.59051737217149, 'heme_age_gap': 29.59051737217149}


## 3. Export Model Metrics

In [6]:
# Restructure metrics for frontend - FLAT FORMAT
metrics_export = []

# Frontend display name mapping
display_names = {
    'liver': 'Liver',
    'kidney': 'Kidney',
    'cardio_metabolic': 'Cardio-Metabolic',
    'immune': 'Immune',
    'hematologic': 'Hematologic'
}

for organ in organs:
    if organ in metrics:
        organ_data = metrics[organ]
        
        # Calculate improvement
        linear_mae = organ_data['linear']['test']['mae']
        gb_mae = organ_data['gradient_boosting']['test']['mae']
        improvement = ((linear_mae - gb_mae) / linear_mae) * 100
        
        # Flat structure for frontend
        metrics_export.append({
            "organ": display_names.get(organ, organ.replace('_', ' ').title()),
            "mae_linear": round(linear_mae, 2),
            "mae_nonlinear": round(gb_mae, 2),
            "improvement_pct": round(improvement, 1),
            "r2": round(organ_data['gradient_boosting']['test']['r2'], 2)
        })

# Save
output_path = output_dir / "metrics_summary.json"
with open(output_path, 'w') as f:
    json.dump(metrics_export, f, indent=2)

print(f"✓ Exported metrics_summary.json")
print(f"✓ Sample metric: {metrics_export[0]}")

✓ Exported metrics_summary.json
✓ Sample metric: {'organ': 'Liver', 'mae_linear': 13.82, 'mae_nonlinear': 12.56, 'improvement_pct': 9.1, 'r2': 0.32}


## 4. Export Correlations

In [7]:
# Calculate correlation matrix
corr_matrix = df[gap_cols].corr()

# Frontend organ labels (short names)
frontend_labels = ['Liver', 'Kidney', 'Cardio', 'Immune', 'Heme']

correlations_export = {
    "labels": frontend_labels,
    "matrix": corr_matrix.values.tolist()
}

# Save
output_path = output_dir / "correlations.json"
with open(output_path, 'w') as f:
    json.dump(correlations_export, f, indent=2)

print(f"✓ Exported correlations.json")
print("\nCorrelation Matrix:")
print(corr_matrix)

✓ Exported correlations.json

Correlation Matrix:
                          liver_age_gap  kidney_age_gap  \
liver_age_gap                  1.000000        0.737627   
kidney_age_gap                 0.737627        1.000000   
cardio_metabolic_age_gap       0.556032        0.545202   
immune_age_gap                 0.734313        0.708347   
hematologic_age_gap            0.706458        0.663790   

                          cardio_metabolic_age_gap  immune_age_gap  \
liver_age_gap                             0.556032        0.734313   
kidney_age_gap                            0.545202        0.708347   
cardio_metabolic_age_gap                  1.000000        0.632756   
immune_age_gap                            0.632756        1.000000   
hematologic_age_gap                       0.595762        0.786192   

                          hematologic_age_gap  
liver_age_gap                        0.706458  
kidney_age_gap                       0.663790  
cardio_metabolic_age_gap      

## 5. Export Pseudo-Longitudinal Trajectories

In [8]:
# Define age bins
age_bins = [18, 30, 40, 50, 60, 70, 80]
age_bin_labels = [f"{age_bins[i]}-{age_bins[i+1]}" for i in range(len(age_bins)-1)]

# Calculate trajectories
trajectories_export = {
    "age_bins": age_bin_labels,
    "organs": {},
    "date_generated": datetime.now().isoformat()
}

for organ in organs:
    gap_col = f"{organ}_age_gap"
    
    mean_gaps = []
    std_gaps = []
    n_individuals = []
    
    for i in range(len(age_bins)-1):
        mask = (df['AGE'] >= age_bins[i]) & (df['AGE'] < age_bins[i+1])
        subset = df[mask][gap_col]
        
        mean_gaps.append(float(subset.mean()) if len(subset) > 0 else None)
        std_gaps.append(float(subset.std()) if len(subset) > 0 else None)
        n_individuals.append(int(len(subset)))
    
    trajectories_export["organs"][organ] = {
        "display_name": organ.replace('_', ' ').title(),
        "mean_gaps": mean_gaps,
        "std_gaps": std_gaps,
        "n_individuals": n_individuals
    }

# Save
output_path = output_dir / "trajectories.json"
with open(output_path, 'w') as f:
    json.dump(trajectories_export, f, indent=2)

print(f"✓ Exported trajectories.json")

✓ Exported trajectories.json


## 6. Export Clustering Results (if available)

In [9]:
# Generate clusters based on K-means results from notebook 05
print("✓ Generating cluster summary...")

# Get unique clusters
unique_clusters = sorted(df['cluster'].unique())

# Cluster names and descriptions based on K-means analysis (notebook 05)
# These reflect the actual aging patterns discovered by the algorithm
cluster_info = {
    0: {
        "name": "Healthy Elderly",
        "description": "Older individuals (70+ yrs) with decelerated aging across all organ systems. Biological age younger than chronological age."
    },
    1: {
        "name": "Accelerated Young",
        "description": "Young individuals (20-30 yrs) showing premature aging across multiple organs. Requires early intervention."
    },
    2: {
        "name": "Moderate Acceleration",
        "description": "Middle-aged individuals (40-50 yrs) with moderate accelerated aging, particularly in immune and hematologic systems."
    },
    3: {
        "name": "Balanced Seniors",
        "description": "Senior individuals (60+ yrs) aging at near-expected rates, with minor deceleration in some systems."
    }
}

clusters_export = []

# Cluster summaries - FRONTEND FLAT FORMAT
for cluster_id in unique_clusters:
    mask = df['cluster'] == cluster_id
    cluster_df = df[mask]
    
    info = cluster_info.get(cluster_id, {"name": f"Cluster {cluster_id}", "description": ""})
    
    clusters_export.append({
        "id": int(cluster_id),
        "name": info["name"],
        "percentage": round(100 * len(cluster_df) / len(df), 0),
        "description": info["description"]
    })

# Save
output_path = output_dir / "clusters.json"
with open(output_path, 'w') as f:
    json.dump(clusters_export, f, indent=2)

print(f"✓ Exported clusters.json")
cluster_dist = [f"{c['name']}: {c['percentage']}%" for c in clusters_export]
print(f"✓ Cluster distribution: {cluster_dist}")

✓ Generating cluster summary...
✓ Exported clusters.json
✓ Cluster distribution: ['Healthy Elderly: 17.0%', 'Accelerated Young: 25.0%', 'Moderate Acceleration: 30.0%', 'Balanced Seniors: 27.0%']


## 7. Export Feature Importance (Placeholder)

In [10]:
# Feature importance mapping (to be filled with actual importance values)
# This is a placeholder - actual values should come from trained models

feature_info = {
    "liver": {
        "features": [
            {"name": "LBXSASSI", "display_name": "AST (Aspartate Aminotransferase)", 
             "importance": 0.25, "description": "Marker of liver cell damage",
             "direction": "Higher values → older predicted age"},
            {"name": "LBXSAL", "display_name": "Albumin",
             "importance": 0.18, "description": "Reflects liver synthetic function",
             "direction": "Lower values → older predicted age"},
            {"name": "LBXSGTSI", "display_name": "GGT (Gamma-Glutamyl Transferase)",
             "importance": 0.15, "description": "Elevated in liver disease",
             "direction": "Higher values → older predicted age"},
        ]
    },
    "kidney": {
        "features": [
            {"name": "LBXSCR", "display_name": "Creatinine",
             "importance": 0.30, "description": "Primary marker of kidney function",
             "direction": "Higher values → older predicted age"},
            {"name": "LBXSBU", "display_name": "BUN (Blood Urea Nitrogen)",
             "importance": 0.22, "description": "Waste product filtered by kidneys",
             "direction": "Higher values → older predicted age"},
        ]
    },
    "cardio_metabolic": {
        "features": [
            {"name": "BPXSY1", "display_name": "Systolic Blood Pressure",
             "importance": 0.28, "description": "Increases with vascular aging",
             "direction": "Higher values → older predicted age"},
            {"name": "LBXGH", "display_name": "HbA1c (Glycohemoglobin)",
             "importance": 0.20, "description": "Long-term glucose control marker",
             "direction": "Higher values → older predicted age"},
        ]
    },
    "immune": {
        "features": [
            {"name": "LBXWBCSI", "display_name": "White Blood Cell Count",
             "importance": 0.25, "description": "Overall immune cell count",
             "direction": "Complex relationship with age"},
            {"name": "LBXLYPCT", "display_name": "Lymphocyte Percentage",
             "importance": 0.20, "description": "Adaptive immunity marker",
             "direction": "Lower values → older predicted age"},
        ]
    },
    "hematologic": {
        "features": [
            {"name": "LBXHGB", "display_name": "Hemoglobin",
             "importance": 0.22, "description": "Oxygen-carrying protein",
             "direction": "Lower values → older predicted age"},
            {"name": "LBXRDW", "display_name": "Red Cell Distribution Width",
             "importance": 0.18, "description": "Variability in RBC size",
             "direction": "Higher values → older predicted age"},
        ]
    }
}

# Export each organ's feature importance
for organ, data in feature_info.items():
    feature_export = {
        "organ": organ,
        "display_name": organ.replace('_', ' ').title(),
        "model": "HistGradientBoosting",
        "features": data["features"],
        "note": "Feature importance values are illustrative. Run SHAP analysis for actual values.",
        "date_generated": datetime.now().isoformat()
    }
    
    output_path = output_dir / "feature_importance" / f"{organ}.json"
    with open(output_path, 'w') as f:
        json.dump(feature_export, f, indent=2)
    
    print(f"✓ Exported feature_importance/{organ}.json")

✓ Exported feature_importance/liver.json
✓ Exported feature_importance/kidney.json
✓ Exported feature_importance/cardio_metabolic.json
✓ Exported feature_importance/immune.json
✓ Exported feature_importance/hematologic.json


## 8. Export Individual Profiles (Sample)

In [11]:
# Individual profiles are embedded in age_gaps.json, so we don't need separate files
print("✓ Skipping individual profile export (data is in age_gaps.json)")

✓ Skipping individual profile export (data is in age_gaps.json)


## Summary

In [12]:
print("\n" + "="*60)
print("✅ EXPORT COMPLETE - FRONTEND FORMAT")
print("="*60)
print(f"\nOutput directory: {output_dir}")
print("\n📁 Generated files:")
print("  ✓ age_gaps.json (with seqn, sex M/F, cluster, cardio/heme names)")
print("  ✓ metrics_summary.json (flat format with mae_linear, mae_nonlinear, r2, improvement_pct)")
print("  ✓ correlations.json (with short organ names)")
print("  ✓ clusters.json (flat array format)")
print("  ✓ feature_importance/ (5 files)")
print(f"\n📊 Statistics:")
print(f"  • {len(age_gaps_export['data'])} individuals")
print(f"  • {len(metrics_export)} organs")
print(f"  • {len(clusters_export)} clusters")
print("\n🚀 Next steps:")
print("  1. cd frontend/longevity---organ-aging-analysis---vitalist")
print("  2. npm run dev")
print("  3. Open http://localhost:5173")
print("\n" + "="*60)


✅ EXPORT COMPLETE - FRONTEND FORMAT

Output directory: C:\Users\bastien\Documents\TAF\Hackathon\Vitalist\frontend\longevity---organ-aging-analysis---vitalist\public\data

📁 Generated files:
  ✓ age_gaps.json (with seqn, sex M/F, cluster, cardio/heme names)
  ✓ metrics_summary.json (flat format with mae_linear, mae_nonlinear, r2, improvement_pct)
  ✓ correlations.json (with short organ names)
  ✓ clusters.json (flat array format)
  ✓ feature_importance/ (5 files)

📊 Statistics:
  • 531 individuals
  • 5 organs
  • 4 clusters

🚀 Next steps:
  1. cd frontend/longevity---organ-aging-analysis---vitalist
  2. npm run dev
  3. Open http://localhost:5173

